In [ ]:
# !wget https://huggingface.co/datasets/malaysia-ai/wikipedia/resolve/main/wikipedia-2023-10-01.jsonl

In [ ]:
import json
from tqdm import tqdm

partitions = []
s = 512

with open('wikipedia-2023-10-01.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        l = l.split()
        l = [l[i: i + s] for i in range(0, len(l), s)]
        l = [l_ for l_ in l if len(l_) > 100]
        l = [' '.join(l_) for l_ in l]
        partitions.extend(l)
        
len(partitions)

In [ ]:
partitions[0]

In [ ]:
prompts = []
for a in partitions:
    p = f"""
paragraph = ```
{a}
```

generate 100 words persona based on paragraph above, example generated persona like below,

```
Zara is a strong and patient woman but still has emotional problems due to her unhappy divorce relationship. However, she remains optimistic and persistent in maintaining her dignity and wants to be an example for other women who face the same challenges. She also likes to seek advice and support from her friends.
```
Or,
```
Azmi bin Ahmad is a strong and persistent individual in computer science. However, he also faces emotional challenges due to his unending divorce relationships. Despite this, Azmi remains optimistic and determined to maintain his dignity and hopes to be an example for other individuals facing similar challenges.
```
Or,
```
Panglima Awang, also known as Enrique of Malacca, is a brave and wise Malay hero. As a war captive who was taken from Melaka to Lisbon, Portuguese, he had struggled with strength and intelligence to survive on foreign lands. Enrique possessed an unassuming spirit and outstanding language skills, enabling him to communicate with various nations.
```
    """
    prompts.append((p.strip(), a))

In [ ]:
# !rm -rf generate-persona-wikipedia
!mkdir generate-persona-wikipedia

In [ ]:
import requests
import os

def answer(q, i):
    filename = os.path.join('generate-persona-wikipedia', f'{i}.json')
    filename_fail = os.path.join('generate-persona-wikipedia', f'{i}.json.fail')
    
    if os.path.exists(filename) and os.path.getsize(filename) > 200:
        return
    
    if os.path.exists(filename_fail):
        return
    
    for _ in range(3):
        try:
            headers = {
                'accept': 'application/json',
                'Content-Type': 'application/json',
            }
            
            messages = [
                {'role': 'user', 'content': q[0]}
            ]

            json_data = {
                'model': 'mallam-small-2.5',
                'messages': messages,
                'temperature': 0.9,
                'top_p': 0.8,
                'top_k': 50,
                'max_tokens': 512,
            }

            response = requests.post('http://localhost:8000/v1/chat/completions', 
                                     headers=headers, json=json_data)
            if 'maximum context length' in response._content.decode():
                break
            r = response.json()
            r = r['choices'][0]['message']['content']
            if len(r):
                with open(filename, 'w') as fopen:
                    json.dump({'r': r, 'context': q[1]}, fopen)
            
            return
        except:
            pass
    
    with open(filename_fail, 'w') as fopen:
        fopen.write('a')

In [ ]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [ ]:
from threading import Thread
from queue import Queue

queue = Queue()
urls = [(q, no) for no, q in enumerate(prompts)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
answer(*urls[0])

In [ ]:
answer(*urls[-1])

In [ ]:
answer(*urls[-3])

In [ ]:
from tqdm import tqdm

max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()